# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,-0.900796,-0.843627,-1.120513
c,-0.940196,0.710438,0.019585
e,-1.328859,0.508177,0.461886
f,1.167459,-2.105378,-0.991711
h,0.693860,0.040233,-0.090509


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [3]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,-0.900796,-0.843627,-1.120513,bar,False
c,-0.940196,0.710438,0.019585,bar,False
e,-1.328859,0.508177,0.461886,bar,False
f,1.167459,-2.105378,-0.991711,bar,True
h,0.693860,0.040233,-0.090509,bar,True


Adding new rows is also simple. Below we include three extra empty rows:

In [4]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,-0.900796,-0.843627,-1.120513,bar,False
b,NaN,NaN,NaN,NaN,NaN
c,-0.940196,0.710438,0.019585,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-1.328859,0.508177,0.461886,bar,False
f,1.167459,-2.105378,-0.991711,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,0.693860,0.040233,-0.090509,bar,True


# Your Turn Here

Do you still remember how to index a row/column?

In [10]:
#### index row 'c' below
df2.loc['c']

one      -0.940196
two       0.710438
three    0.0195851
four           bar
five         False
Name: c, dtype: object

In [12]:
#### index column 'two' below
df2['two']

a   -0.843627
b         NaN
c    0.710438
d         NaN
e    0.508177
f   -2.105378
g         NaN
h    0.040233
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [13]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [14]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [15]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,-0.843627
c,NaN,0.710438
e,NaN,0.508177
f,1.167459,-2.105378
h,0.693860,0.040233


In [16]:
b = df[['one','two','three']]
b

,one,two,three
a,-0.900796,-0.843627,-1.120513
c,-0.940196,0.710438,0.019585
e,-1.328859,0.508177,0.461886
f,1.167459,-2.105378,-0.991711
h,0.693860,0.040233,-0.090509


In [17]:
a + b

,one,three,two
a,NaN,NaN,-1.687254
c,NaN,NaN,1.420877
e,NaN,NaN,1.016355
f,2.334918,NaN,-4.210757
h,1.387721,NaN,0.080466


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [18]:
a['one'].dropna()

f    1.167459
h    0.693860
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [19]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,-0.843627
c,0.710438
e,0.508177
f,-2.105378
h,0.040233


In [20]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,-1.687254
c,NaN,1.420877
e,NaN,1.016355
f,2.334918,-4.210757
h,1.387721,0.080466


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [21]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,-1.687254
c,NaN,1.420877
e,NaN,1.016355
f,2.334918,-4.210757
h,1.387721,0.080466


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [22]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [23]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [24]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [25]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [26]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,0.285634,0.978241,NaN
b,NaN,-1.201161,-0.266180
c,-0.601046,0.221905,NaN
d,0.021808,NaN,-0.025158
e,-2.333339,-0.570434,-0.074011


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [27]:
#### insert your code here
rand_df['one'] = rand_df['one'].fillna(1)
rand_df


,one,two,three
a,0.285634,0.978241,NaN
b,1.000000,-1.201161,-0.266180
c,-0.601046,0.221905,NaN
d,0.021808,NaN,-0.025158
e,-2.333339,-0.570434,-0.074011


In [28]:
rand_df['two'] = rand_df['two'].fillna(method = 'ffill')
rand_df

,one,two,three
a,0.285634,0.978241,NaN
b,1.000000,-1.201161,-0.266180
c,-0.601046,0.221905,NaN
d,0.021808,0.221905,-0.025158
e,-2.333339,-0.570434,-0.074011


In [29]:
rand_df['three'] = rand_df['three'].fillna(method = 'bfill')
rand_df

,one,two,three
a,0.285634,0.978241,-0.266180
b,1.000000,-1.201161,-0.266180
c,-0.601046,0.221905,-0.025158
d,0.021808,0.221905,-0.025158
e,-2.333339,-0.570434,-0.074011


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [30]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,0.143518,-0.340075,1.062076
b,1.974593,NaN,NaN
c,0.626310,NaN,1.945241
d,0.914312,NaN,0.172010
e,-0.235736,NaN,-0.078163


In [31]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [32]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A    0.684600
B   -0.340075
C    0.775291
dtype: float64

In [33]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,0.143518,-0.340075,1.062076
b,1.974593,-0.340075,0.775291
c,0.626310,-0.340075,1.945241
d,0.914312,-0.340075,0.172010
e,-0.235736,-0.340075,-0.078163


In [34]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.